# Juliaで100本ノック(51-76)

## 準備

In [ ]:
ENV["COLUMNS"]=240  # 描画する表の列数を増やす
ENV["LINES"]=10  # 行の数は制限（問題の指示とは異なるので好みに合わせて修正）

using Pkg

Pkg.add("DataFrames")
Pkg.add("DataFramesMeta")
Pkg.add("LibPQ")
Pkg.add("StatsBase")
Pkg.add("ScikitLearn")

using DataFrames
using DataFramesMeta
using LibPQ
using StatsBase
using Statistics
using Dates
using Random
using ScikitLearn

In [ ]:
@sk_import preprocessing: (LabelBinarizer, StandardScaler, MinMaxScaler)

## SQLとの接続

In [ ]:
host = "db"
port = ENV["PG_PORT"]
database = ENV["PG_DATABASE"]
user = ENV["PG_USER"]
password = ENV["PG_PASSWORD"]
dsl = "postgresql://$user:$password@$host:$port/$database"
conn = LibPQ.Connection(dsl)

df_customer = DataFrame(execute(conn, "select * from customer"))
df_category = DataFrame(execute(conn, "select * from category"))
df_product = DataFrame(execute(conn, "select * from product"))
df_receipt = DataFrame(execute(conn, "select * from receipt"))
df_store = DataFrame(execute(conn, "select * from store"))
df_geocode = DataFrame(execute(conn, "select * from geocode"));

## 本編

### 051

In [ ]:
# 前問と同じ
@linq df_receipt |>
    select(:receipt_no, :receipt_sub_no, :sales_epoch) |>
    transform(sales_epoch = lpad.(Dates.day.(unix2datetime.(:sales_epoch)), 2, "0")) |>
    first(10)

### 052

In [ ]:
# 匿名関数を作ってその場でelement-wiseに適用。他は既出の要素の組み合わせ。
@linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    transform(sales_flg = (x -> x>2000 ? 1 : 0).(:amount_sum)) |>
    orderby(:customer_id) |>
    first(10)

### 053

In [ ]:
# 別関数で郵便番号の判定
function iftokyo(x::String)
    code = parse(Int, x[1:3])
    if code >= 100 && code <= 209
        return true
    else
        return false
    end
end

In [ ]:
# joinする前にuniquifyして行数が増えないようにする。
@linq df_customer |>
    select(:customer_id, :postal_cd) |>
    transform(postal_flg = iftokyo.(:postal_cd)) |>
    innerjoin(unique(df_receipt, :customer_id), on=:customer_id) |>
    groupby(:postal_flg) |>
    combine(:customer_id => length) |>
    orderby(:postal_flg)


### 054

In [ ]:
# Pythonと違ってdictをそのままmapに使えないし、それにいずれにせよ部分文字列にマッチさせる必要があるので関数に分ける。
# 本当は正規表現使わずに先頭の数文字を比べるだけでよい。
function get_pref(x::String)
    if occursin(r"^埼玉県", x)
        return 11
    elseif occursin(r"^千葉県", x)
        return 12
    elseif occursin(r"^東京都", x)
        return 13
    elseif occursin(r"^神奈川県", x)
        return 14
    end
end

@linq df_customer |>
    select(:customer_id, :address) |>
    transform(address_cd = get_pref.(:address)) |>
    first(10)

### 055

In [ ]:
df_sales_amount = @linq df_receipt |>
    select(:customer_id, :amount) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    orderby(:customer_id);

In [ ]:
_, pct25, pct50, pct75 = nquantile(df_sales_amount.amount_sum, 4)

function pct_group(x::Real)
    if x < pct25
        return 1
    elseif x < pct50
        return 2
    elseif x < pct75
        return 3
    else
        return 4
    end
end

In [ ]:
@linq df_sales_amount |>
    transform(pct_group = pct_group.(:amount_sum)) |>
    first(10)

### 056

In [ ]:
# ややこしい処理ではないが分けたほうがわかりやすい
function get_ageclass(x)
    ageclass = Int(floor(x÷10)*10)
    return min(ageclass, 60)
end

@linq df_customer |>
    select(:customer_id, :birth_day, :age) |>
    transform(age = get_ageclass.(:age)) |>
    first(10)

### 057

In [ ]:
function get_agegender(gender_cd::String, age::Int64)
    return string(gender_cd, age)
end

@linq df_customer |>
    transform(age = get_ageclass.(:age)) |>
    transform(age_gender = get_agegender.(:gender_cd, :age)) |>
    select(:customer_id, :birth_day, :age, :age_gender) |>
    first(10)

### 058

In [ ]:
# StatModels.jlを使う手もあるが、ここではScikitLearnを呼んでみることにする。どちらにしてもArrayになってしまうのでwrapperを用意。

function get_onehot(df::DataFrame, col::Symbol)
    binalizer = LabelBinarizer()
    mapper = DataFrameMapper([(col, binalizer)])
    onehotdf = DataFrame(Int.(fit_transform!(mapper, copy(df))))
    rename!(onehotdf, [string(col, "_", each) for each in binalizer.classes_])
    return onehotdf
end

@linq hcat(df_customer[:, :customer_id], get_onehot(df_customer, :gender_cd)) |>
    first(10)

### 059

In [ ]:
# 標本標準偏差による標準化ならz-scoreを求めるのと同値
@linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    transform(amount_ss = zscore(:amount_sum)) |>
    orderby(:customer_id) |>
    first(10)

### 060

In [ ]:
# 今度はStatsBaseを使って正規化。Intの入力を受け付けないためFloatに変換しておく。
@linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    transform(amount_mm = (x -> standardize(UnitRangeTransform, Float64.(x), dims=1))(:amount_sum)) |>
    orderby(:customer_id) |>
    first(10)

### 061

In [ ]:
# Python版と微妙に数字が合わないのは計算や定数定義の桁数の問題か？
@linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    transform(amount_log10 = log10.(:amount_sum)) |>
    orderby(:customer_id) |>
    first(10)

### 062

In [ ]:
# 同上
@linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    transform(amount_loge = log.(:amount_sum)) |>
    orderby(:customer_id) |>
    first(10)

### 063

In [ ]:
@linq df_product |>
    transform(unit_profit = :unit_price .- :unit_cost) |>
    first(10)

### 064

In [ ]:
# いまいちスマートではないが妥協
df_tmp = @linq df_product |>
    transform(unit_profit_rate = (:unit_price .- :unit_cost)./:unit_price);
mean(skipmissing(df_tmp[:, :unit_profit_rate]))

### 065

In [ ]:
@linq df_product |>
    transform(new_price = floor.(:unit_cost ./ 0.7)) |>
    transform(new_profit_rate = (:new_price .- :unit_cost) ./ :new_price) |>
    first(10)

### 066

In [ ]:
# Juliaのroundも.5は偶数方向に丸められる
@linq df_product |>
    transform(new_price = round.(:unit_cost ./ 0.7)) |>
    transform(new_profit_rate = (:new_price .- :unit_cost) ./ :new_price) |>
    first(10)

### 067

In [ ]:
@linq df_product |>
    transform(new_price = ceil.(:unit_cost ./ 0.7)) |>
    transform(new_profit_rate = (:new_price .- :unit_cost) ./ :new_price) |>
    first(10)

### 068

In [ ]:
@linq df_product |>
    transform(price_tax = floor.(:unit_price .* 1.1)) |>
    first(10)

### 069

In [ ]:
df_tmp1 = @linq df_receipt |>
    select(:customer_id, :amount) |>
    groupby(:customer_id) |>
    combine(:amount => sum)
rename!(df_tmp1, ["customer_id", "amount_x"]);

In [ ]:
df_tmp2 = @linq innerjoin(df_receipt, df_product, on=:product_cd) |>
    where(:category_major_cd .== "07") |>
    select(:customer_id, :amount) |>
    groupby(:customer_id) |>
    combine(:amount => sum)
rename!(df_tmp2, ["customer_id", "amount_y"]);

In [ ]:
@linq innerjoin(df_tmp1, df_tmp2, on=:customer_id) |>
    transform(rate_07 = :amount_y ./ :amount_x) |>
    orderby(:customer_id) |>
    first(10)

### 070

In [ ]:
# じつはselectの中でも加工できる
@linq innerjoin(df_receipt, df_customer, on=:customer_id) |>
    select(:customer_id,
           sales_ymd = Date.(string.(:sales_ymd), "yyyymmdd"),
           application_date = Date.(:application_date, "yyyymmdd")) |>
    transform(elapsed_date = :sales_ymd .- :application_date) |>
    orderby(:customer_id) |>
    last(10)
# ISSUE: Python版と結果が一致しない（ソートが異なる）

### 071

In [ ]:
# 月数の定義があいまいなためスキップ。経過日数÷30でいいなら上の回答ほぼそのまま。

### 072

In [ ]:
# 月数の定義があいまいなためスキップ。経過日数÷30でいいなら上の回答ほぼそのまま。

### 073

In [ ]:
@linq innerjoin(df_receipt, df_customer, on=:customer_id) |>
    select(:customer_id,
           sales_ymd = Date.(string.(:sales_ymd), "yyyymmdd"),
           application_date = Date.(:application_date, "yyyymmdd")) |>
    transform(elapsed_second = convert.(Dates.Second, :sales_ymd .- :application_date)) |>
    orderby(:customer_id) |>
    last(10)

### 074

In [ ]:
# Dates.dayofweekは月曜日が1なので差し引いてやる
@linq df_receipt |>
    select(:customer_id,
           sales_ymd = Date.(string.(:sales_ymd), "yyyymmdd")) |>
    transform(elapsed_weekday = Day.(Dates.dayofweek.(:sales_ymd) .-1)) |>
    transform(monday = :sales_ymd .- :elapsed_weekday) |>
    first(10)

### 075

In [ ]:
# pandasと違ってそのままサンプリングできないので取り出すインデックスの配列を経由する
# ScikitLearnにもsample()があるのでモジュールを指定
# MLDataPatternモジュールを使えば直接サンプリングできるが、ArrayとDataFrameの間の変換が必要でかえって煩雑
n = nrow(df_receipt)
index = StatsBase.sample(1:n, n÷100, replace=false)
first(df_receipt[index, :], 10)